## Notebook to demonstrate the simplicity of saving data in QKIT
The `Data` class is used to write save and put data into `h5` files for each readout. Here we see how we can manipulate and plot the readout data using h5 files and `data` class.

### First import the QKIT store module

In [41]:
import qkit
from qkit.storage import store
qkit.cfg['datadir'] = r'C:\Users\weideslab\Documents\QkitLearn\data'
qkit.cfg['run_id']='basic_data_storage'
qkit.cfg['user']='wridhdhi'
qkit.cfg['fid_scan_datadir']=True # scans the data directory for all h5 files
qkit.cfg['fid_scan_hdf']=True # scans all the files and extracts attributes
qkit.cfg['fid_init_viewer']=True # creates a dictionary of all those files


... some imports to create some random data ...

In [42]:
## for random data
from numpy.random import rand
from numpy import linspace,arange
import time
## number of points
nop = 101

## 1. Create a data file object
The file name will be ** prepended by a unique 6 character identifier (UUID) ** 
and the file saved in a standard path below automatically. Also we can use the `fid` class to access the existing data in the following directory :

`qkit.cfg['datadir']`

Below this new file is opened in appending mode. 

In [43]:
h5d = store.Data(name='NewFancyData',mode = "a")

`h5d`  is now an empty hdf5 file, holding only some qkit structure, this is initializing the hdf5 file and it by default adds an identifier, and stored in `datadir` location. Now our job is to put data in this hdf5 file. 

#### How to get the file path:
use the `Data.get_filepath()` function to get the absolute path of the h5 file.

In [44]:
print(h5d.get_filepath())

C:\Users\weideslab\Documents\QkitLearn\data\basic_data_storage\wridhdhi\QRT3CL_NewFancyData\QRT3CL_NewFancyData.h5


#### Now use qviewkit to open and view the file
_Hint: in a fully started qkit environment (qkit.start()) This can be done by qkit.fid.view("UUID")_ ,

In [48]:
qkit.start()
import qkit.core.lib.file_service.file_info_database as fid


Starting QKIT framework ... -> qkit.core.startup
Loading module ... S10_logging.py
Loading module ... S14_setup_directories.py
Loading module ... S16_available_modules.py
Loading module ... S20_check_for_updates.py
Loading module ... S25_info_service.py
Loading module ... S30_qkit_start.py
Loading module ... S65_load_RI_service.py
Loading module ... S70_load_visa.py
Loading module ... S80_load_file_service.py
Loading module ... S85_init_measurement.py
Loading module ... S98_started.py
Loading module ... S99_init_user.py
Initialized the file info database (qkit.fid) in 0.014 seconds.


In [49]:
f=fid.fid()
f.view('QRT3CL')

Initialized the file info database (qkit.fid) in 0.017 seconds.


## 1a. Comments
Since the file is blank let's add some comments. 

**function to add comment to the basic hdf `data` folder** :

`Data.add_comment('comment',folder='data')`
options
 - comment (mandatory)
 - folder='data' | 'analysis' (optional, default is "data") 

In [50]:
h5d.add_comment("New data has been created ....")

## 2. Now we get to the important stuff: 

 1. first we have to create the objects which are later used to store the data
 2.  then we can add data 

#### a. A coordinate is a 1-dim vector with bias values, e.g. set current, frequencies, etc.
Lets add cooredinates to out file,its like defining **an axis**, a value you would measure. but in qkit it holds units, and comments as well. Note we are just defining the coordinate not actually adding any data. We are spcifying inner structure of the `data` file in h5 format. 

In [52]:
# add_coordinate()    <- for measurement boundaries/steps
# options: name (mandatory)
#        : unit = "" (optional, default is "a.u.")
#        : comment = "" (optional, default is "")
#        : folder='data' | 'analysis' (optional, default is "data") 
f_co = h5d.add_coordinate('frequency', unit = "Hz",   comment = "VNA frequency scan")
I_co = h5d.add_coordinate('current',   unit = "A",    comment = "magnetic field current")
P_co = h5d.add_coordinate('power',     unit = "dBm",  comment = "microwave power")

#### b. A value_vector ie is a 1-dim vector measured values
Its just values in of a coordinate described above. So you can assgn some coordinate or not using the `x` attribute. 


In [53]:
# add_value_vector()    <- for measurement data
# options: name (mandatory)
#        : x = X  (optional) coordinate vector in x direction, default: None
#        : unit = "" (optional, default is "a.u.")
#        : comment = "" (optional, default is "")
#        : folder='data' | 'analysis' (optional, default is "data") 

T_vec = h5d.add_value_vector('temperature', x = None, 
                             unit = "K", comment = "save temperature values") 
Tc_vec = h5d.add_value_vector('critical_temperature', x = I_co, 
                              unit = "K", folder='analysis' ,comment = "save temperature values")

#### c. A value_matrix is a 2-dim tuple with two cooardinates with measurement values
Imagine a $n x n$ array of `value-vectors` , this you can definitely plot along both the axes. 

In [54]:
# add_value_matrix()    <- for measurement data
# convention: the last coordiante should be the one with the fastest changes:
#             e.g.  for a VNA scan x= magnetic field y= transmission frequency
# 
# options: name (mandatory)
#        : x = X  (optional) coordinate vector in x direction, default: None
#        : y = Y  (mandatory) coordinate vector in y direction / fastest changes
#        : unit = "" (optional, default is "a.u.")
#        : comment = "" (optional, default is "")
#        : folder='data' | 'analysis' (optional, default is "data") 

amp_mx = h5d.add_value_matrix('amplitude', x = I_co , y = f_co, 
                              unit = "V", comment = "magic data")
pha_mx = h5d.add_value_matrix('phase',     x = I_co , y = f_co, 
                              unit = "rad", comment = "more magic data!")


#### d. A value_box is a 3-dim tuple with three cooardinates with measurement values
A $n x n x n$ array of data which you can do a color plot, and use against 2 axes. 

In [55]:
# add_value_box()    <- for measurement data
# options: name (mandatory)
#        : x = X  (optional) coordinate vector in x direction, default: None
#        : y = Y  (optional) coordinate vector in y direction
#        : z = Z  (mandatory) coordinate vector in y direction /  fastest changes
#        : unit = "" (optional, default is "a.u.")
#        : comment = "" (optional, default is "")
#        : folder='data' | 'analysis' (optional, default is "data") 

amp_bx = h5d.add_value_box('amplitude', x = I_co , y = f_co, z= P_co, 
                           unit = "V", comment = "magic data")
pha_bx = h5d.add_value_box('phase',     x = I_co , y = f_co, z= P_co, 
                           unit = "rad", comment = "more magic data!")




#### e. For free text, like settings or commments there is a special dataset which can be appended to add data

In [16]:
#string array 
#add_textlist()
#options: name (mandatory)
#       : comment = ""   (optional)
#       : folder="data"  (optional)
# use the append method to add the text
settings = h5d.add_textlist("settings",comment = "my settings")
settings.append(u"vnapower = 10dBm")
settings.append(u"fridge attenuation=50db\n data jumps like Van Halen.")


#### f. If one want to plot a arbitrary dataset vs another arbitrary dataset use a view
A view is a plot basically. 

In [17]:
### Add a view on Data:

TvsTc_view = h5d.add_view("f_vs_I", x = f_co, y = I_co)

TvsTc_view.add(x=T_vec,y=Tc_vec)

Laspect_view = h5d.add_view("locked_aspect", x = f_co, y = f_co, 
                            view_params={'aspect':1.0, 'bgcolor':(100,200,100)})



### g. Finally add some data to the file
There are two functions to do so:
  * append(data)  <- this is the most frequently used method, it does what it says.
  * add(data)     <- this is used to set a dataset of lenght N to the data with length N

lets make an example:

In [18]:
# now we add the coordinate data to the file
fs = linspace(1e9,5e9,nop)
Is = linspace(0e-3,10e-3,nop)


f_co.add(fs)
I_co.add(Is)

for i in arange(nop):
    #time.sleep(10)
    amp = rand(nop)
    pha = rand(nop)
    amp_mx.append(amp)
    pha_mx.append(pha)
    T_vec.append(float(rand(1)))
    Tc_vec.append(float(rand(1)))


### Close the file at the end

In [19]:
h5d.close_file()

In [27]:
qkit.start()

Starting QKIT framework ... -> qkit.core.startup
Loading module ... S10_logging.py
Loading module ... S14_setup_directories.py
Loading module ... S16_available_modules.py
Loading module ... S20_check_for_updates.py
Loading module ... S25_info_service.py
Loading module ... S30_qkit_start.py
Loading module ... S65_load_RI_service.py
Loading module ... S70_load_visa.py
Loading module ... S80_load_file_service.py
Loading module ... S85_init_measurement.py
Loading module ... S98_started.py
Loading module ... S99_init_user.py


In [21]:
qkit.cfg


{'qkitdir': 'C:\\qkit\\qkit',
 'coredir': 'C:\\qkit\\qkit\\core',
 'logdir': 'C:\\qkit\\qkit\\logs',
 'execdir': 'C:\\qkit\\qkit',
 'rootdir': 'C:\\qkit\\qkit',
 'tempdir': 'C:\\Users\\WEIDES~2\\AppData\\Local\\Temp',
 'datadir': 'C:\\Users\\weideslab\\Documents\\QkitLearn\\data',
 'instruments_dir': 'C:\\qkit\\qkit\\drivers',
 'user_instruments_dir': None,
 'datafolder_structure': 2,
 'plot_engine': 'qkit.gui.qviewkit.main',
 'file_log_level': 'INFO',
 'stdout_log_level': 'WARNING',
 'qtlab': False,
 'run_id': 'basic_data_storage',
 'user': 'wridhdhi',
 'maintain_logfiles': True,
 'blocked_modules': [],
 'check_for_updates': False,
 'load_info_service': True,
 'info_host': 'localhost',
 'info_port': 5606,
 'qt_compatible': False,
 'load_ri_service': False,
 'load_visa': False,
 'fid_scan_datadir': True,
 'fid_restrict_to_userdir': False,
 'fid_scan_hdf': False,
 'fid_init_viewer': True,
 'startdir': False,
 'startscript': False,
 'exitscript': False}

In [58]:
import qkit.core.lib.file_service.file_info_database as fid
f=fid.fid()


Initialized the file info database (qkit.fid) in 0.013 seconds.


**Remmember** the fid object can view the file, using qkit view lets see the structure.
The fid functions were:
- `fid.show(True)`
- `fid.view('xxxxxx')`
- `fid.h5_db` and the other databases are attributres as dictionaries. 

In [61]:
f.view('QRHPZG') #opens qkitview for detailed viewing


In [60]:
f.show(True)

,time,datetime,run,name,user,rating
QRHPZG,1618297036,2021-04-13 07:57:16,basic_data_storage,NewFancyData,wridhdhi,
QRHPV9,1618296885,2021-04-13 07:54:45,NO_RUN,NewFancyData,John_Doe,
QFZ34X,1598951841,2020-09-01 10:17:21,QkitLearn,VNA_tracedata,data,
QFZ37C,1598951928,2020-09-01 10:18:48,QkitLearn,VNA_tracedata,data,
QG4U70,1599220188,2020-09-04 12:49:48,QkitLearn,3D_current_drive_frequency,data,
QG4ULX,1599220725,2020-09-04 12:58:45,QkitLearn,3D_current_drive_frequency,data,
QG4ZN4,1599227248,2020-09-04 14:47:28,QkitLearn,3D_current_drive_frequency,data,
QG4ZOT,1599227309,2020-09-04 14:48:29,QkitLearn,3D_current_drive_frequency,data,
QG4ZRK,1599227408,2020-09-04 14:50:08,QkitLearn,3D_current_drive_frequency,data,
QG4ZXL,1599227625,2020-09-04 14:53:45,QkitLearn,3D_current_drive_frequency,data,


## Remmember
The h5 data is in 3 categroeis/folders :
1. data = folder containing 1D,2D,3D data. 
2. views = another folder containing plots
3. analysis = temperature was added in this, nothing different from data justanother folder

Use the following functions :
### adding stuff :
1. 'add_comment',
2.  'add_coordinate',
3. 'add_fid_param',
4. 'add_textlist',
5. 'add_value_box',
6. 'add_value_matrix',
7. 'add_value_vector',
8. 'add_view',

### accessing :
use the `h5d.[folder].[variable]` like `hd5.data.amplitude[:]` to get the amplitude data


### use qkitview to see the file


### analysis
TODO : to write

### then close the file with `hdf.close()`